<a href="https://colab.research.google.com/github/SAULIZA/prediccion_de_ventas/blob/main/Prediccion_de_ventas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PREDICCIONES DE VENTAS Proyecto1- Parte5**
## Creado por: Saúl Iza

* Fecha creado: 18/03/2023
* Fecha modificada: 18/03/2023
* Descripción: proyecto


In [1]:
from google.colab import drive
drive.mount ("/content/drive")

Mounted at /content/drive


In [2]:
#manipulacion de base de datos
import pandas as pd
import numpy as np

* Para la parte 5, regresarán al conjunto de datos original con el objetivo de evitar una fuga de datos. 

In [3]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn import set_config
set_config(display='diagram')

In [4]:
filename ="/content/drive/MyDrive/sales_predictions.csv"
df = pd.read_csv(filename)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [31]:
df["Item_Type"].value_counts()

Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


1. Identifiquen el objetivo (X) y las características (y): Asignen la columna "Item_Outlet_Sales" somo el objetivo y el resto de las variables relevantes como el matriz de características. 

In [9]:
#objetivo
y = df["Item_Outlet_Sales"]
#características reelevantes, dentro de este grupo quitaremos a las columnas
#que son poco reelevantes para la prediccion según las partes 2 3 y 4 del proyecto
# las columnas poco relevantes son Item_Identifier, Item_Weight,Item_Visibility,
# Outlet_Identifier,	Outlet_Establishment_Year,	Outlet_Size,Item_Fat_Content
X = df.drop(columns=["Item_Identifier", "Item_Outlet_Sales", "Item_Weight","Item_Visibility","Outlet_Identifier",	"Outlet_Establishment_Year",	"Outlet_Size", "Item_Fat_Content"])

In [10]:
X.columns

Index(['Item_Type', 'Item_MRP', 'Outlet_Location_Type', 'Outlet_Type'], dtype='object')

2. Realicen un train test split 

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

3. Creen un pipeline de preprocesamiento para preparar el conjunto de datos para el aprendizaje automático.

En la matriz de características tenemos 4 características:
* Item_Type es una característica nominal
* Item_MRP es una característica numérica
* Outlet_Location_Type es una característica ordinal, donde las tiendas con hubicacion tier 1 son las más estatégicas y asi sucesivamente.
* Outlet_type es una característica ordinal, donde el tipo de tienda supermarket Type 1 son los más grandes, asi sucesivamente con los tipo 2 y 3, y finalmente los grocery store son las tiendas más pequeñas.

In [15]:
#El primer paso sería reemplazar las características de tipo ordinal a un número
outlet_location = {"Tier 1" : 1, "Tier 2" : 2, "Tier 3" : 3} 
X_train["Outlet_Location_Type"] = X_train["Outlet_Location_Type"].replace(outlet_location) 
X_test["Outlet_Location_Type"] = X_test["Outlet_Location_Type"].replace(outlet_location)

outlet_type = {"Supermarket Type1" : 1, "Supermarket Type2" : 2, "Supermarket Type3" : 3, "Grocery Store" : 4}
X_train["Outlet_Type"] = X_train["Outlet_Type"].replace(outlet_type) 
X_test["Outlet_Type"] = X_test["Outlet_Type"].replace(outlet_type) 

In [20]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2131 entries, 7503 to 1784
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Item_Type             2131 non-null   object 
 1   Item_MRP              2131 non-null   float64
 2   Outlet_Location_Type  2131 non-null   int64  
 3   Outlet_Type           2131 non-null   int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 83.2+ KB


In [21]:
#Instanciamos column selector
num_selector = make_column_selector(dtype_include = "number", dtype_exclude ="int")
cat_selector = make_column_selector(dtype_include = "object")

In [34]:
# No hay que hacer imputaciones

# scaler y codificacion one hot encoder
scaler = StandardScaler()
ohe_encoder = OneHotEncoder(sparse_output = False, handle_unknown = "ignore")

In [23]:
#pipelines
#parte numérica
numeric_processing = make_pipeline(scaler)
#parte no numérica
cat_processing = make_pipeline(ohe_encoder) 

In [24]:
#creación de tuplas
num_tuple = (numeric_processing, num_selector)
cat_tuple = (cat_processing, cat_selector)

In [25]:
#instanciamos column transform
col_transformer = make_column_transformer(num_tuple,cat_tuple, remainder = "passthrough")

In [26]:
#ajustamos el pipeline en los datos de entrenamiento
col_transformer.fit(X_train)

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f8128de28e0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f8128dddc40>)])

In [27]:
#transformacion de datos
X_train_processed = col_transformer.transform(X_train)
X_test_processed = col_transformer.transform(X_test)

In [28]:
X_train_processed

array([[ 1.82810922,  0.        ,  0.        , ...,  0.        ,
         3.        ,  2.        ],
       [ 0.60336888,  0.        ,  0.        , ...,  0.        ,
         3.        ,  2.        ],
       [ 0.24454056,  0.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       ...,
       [ 1.52302674,  0.        ,  0.        , ...,  0.        ,
         2.        ,  1.        ],
       [-0.38377708,  0.        ,  0.        , ...,  0.        ,
         2.        ,  1.        ],
       [-0.73836105,  0.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ]])

In [29]:
#para hacerlo mas visible
X_train_processed_df = pd.DataFrame(X_train_processed)
X_train_processed_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1.828109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0
1,0.603369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,2.0
2,0.244541,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,-0.952591,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
4,-0.336460,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6387,-0.044657,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0
6388,-1.058907,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
6389,1.523027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
6390,-0.383777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0


In [30]:
X_train_processed_df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
count,6.392000e+03,6392.000000,6392.000000,6392.000000,6392.000000,6392.000000,6392.000000,6392.000000,6392.000000,6392.000000,6392.000000,6392.000000,6392.000000,6392.000000,6392.00000,6392.000000,6392.000000,6392.000000,6392.000000
mean,4.668773e-17,0.074781,0.027378,0.013141,0.075250,0.079318,0.098874,0.148310,0.026439,0.061014,0.108730,0.047247,0.020338,0.006571,0.14174,0.051783,0.019086,2.121402,1.711358
std,1.000078e+00,0.263058,0.163195,0.113889,0.263815,0.270255,0.298516,0.355435,0.160450,0.239374,0.311324,0.212182,0.141164,0.080800,0.34881,0.221607,0.136839,0.809870,1.087075
min,-1.767529e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000,1.000000
25%,-7.638272e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000,1.000000
50%,3.400912e-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,2.000000,1.000000
75%,7.172910e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,3.000000,2.000000
max,1.994559e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,3.000000,4.000000


la columna 0 es Item_MRP
las siguientes corresponten al Item_Type
y las 2 últimas corresponden a Outlet_Location_Type y Outlet_Type